In [1]:
####################################
## A. Librerias
####################################
import os
import time
import json

import datetime
import pandas as pd

from utils.utils import *
from utils.paths import CHROMEDRIVER_PATH, PROJECT_PATH

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

####################################
## B. Funciones
####################################

def get_files(path, extension):
    dir_name  = os.path.join(os.getcwd(), path)
    files     = os.listdir(dir_name)
    
    paths    = []
    for file in files:
        if extension in file:
            paths.append(os.path.join(path,file))

    return paths

def start_har(proxy, name):
    options = {
        'captureHeaders': True, 
        'captureContent': True, 
        'captureBinaryContent': True, 
        'captureCookies': True, 
        'captureHeadersSize': -1, 
        'captureMaxSize': -1, 
        'captureBinaryContentMaxLength': -1 # Tamaño máximo del contenido binario
    }
    proxy.new_har(name, options=options)

ESTAMOS EN --->  darwin
PROJECT_PATH:  /Users/nahumfg/Projects/GitHubProjects/RenacytScraper
CHROME_DRIVER_PATH:  /Users/nahumfg/Projects/GitHubProjects/RenacytScraper/chromedriver/darwin/chromedriver



In [2]:
########################
## C. Ejecución
########################

# 1. Seleccionar modo
HEADLESS = True
PRINT_VIEW = False

# 2. Eliminar todos los archivos de la carpeta
try:
    files = get_files(os.path.join(os.getcwd(),'outputs','jsons','orcid_title'), '.')
    for file in files:
        os.remove(file)
    print('Se eliminaron los archivos de la carpeta')
except:
    pass

Se eliminaron los archivos de la carpeta


In [3]:
######################
## D. Leer los jsons
######################

# 1. Leer los jsons y guardarlos en un dataframe
df_renacyt= pd.read_parquet(os.path.join(os.getcwd(),'outputs','parquets','renacyt.parquet'))
df_renacyt = df_renacyt[df_renacyt['orcid'].notnull()]
df_renacyt = df_renacyt[df_renacyt['orcid'] != 'nan']
df_renacyt = df_renacyt[df_renacyt['orcid'] != 'None']
df_renacyt = df_renacyt[df_renacyt['orcid'] != '']
df_renacyt['orcid'] = df_renacyt['orcid'].astype(str)

# 2. Generar lista de urls a partir de ['ctiVitae']
urls = []
orcirds = df_renacyt['orcid'].tolist()
for orcid in orcirds:
    url = f'https://orcid.org/{orcid}'
    urls.append(url)

#urls = list(set(urls))
print(f'Se generaron {len(urls)} urls')

urls = urls[0:3]
urls

Se generaron 6993 urls


['https://orcid.org/0000-0003-1583-7113',
 'https://orcid.org/0000-0001-9740-5490',
 'https://orcid.org/0000-0002-8301-5536']

In [4]:
######################
## E. Ejecutar
######################

# 1. Iniciar el driver, el servidor proxy y el proxy
driver, server, proxy = get_chrome_driver(chromedriver_path=CHROMEDRIVER_PATH, print_view=PRINT_VIEW, headless=HEADLESS)

# 2. Iterar sobre las urls
i = 0
for url in urls: 
    # A. Iniciar har
    start_har(proxy, 'renacyt_profile')
    
    # B. Ingresar a la url
    driver.get(url)
    print(f'{i+1}. {url}')

    # C. Sí i==0, aceptar cookies
    if i == 0:
        find_element_and_click(driver, 'id', 'onetrust-accept-btn-handler')

    # D. Obtener el har
    time.sleep(3)
    har = proxy.har

    # E. Guardar publicaciones
    j = 0
    for entrie in har.get('log').get('entries'):
        string_content = entrie.get('response').get('content').get('text')
        if '{"title":' in str(string_content) and '"assertionOriginOrcid"' in str(string_content) and '"assertionOriginClientId"' in str(string_content):
            data_list = json.loads(string_content)
            #data_list = str(string_content)
            with open(os.path.join(os.getcwd(),'outputs','jsons','orcid_title','title_' + str(i).zfill(2) + str(j).zfill(2) + '.json'), 'w') as json_file:
                json.dump(data_list, json_file, indent=4)
        j += 1

    # j = 0
    # for entrie in har.get('log').get('entries'):
    #     #string_content = entrie.get('response').get('content').get('text')
    #     #data_list = json.loads(string_content)
    #     data_list = str(entrie)
    #     with open(os.path.join(os.getcwd(),'outputs','jsons','orcid','orcid_' + str(i).zfill(2) + str(j).zfill(2) + '.json'), 'w') as json_file:
    #         json.dump(data_list, json_file, indent=4)
    #         j += 1

    # F. Actualizar el contador
    i += 1
    

Se inicia el servidor en Mac o Linux
Usando el chromedriver_path: /Users/nahumfg/Projects/GitHubProjects/RenacytScraper/chromedriver/darwin/chromedriver
1. https://orcid.org/0000-0003-1583-7113
2. https://orcid.org/0000-0001-9740-5490
3. https://orcid.org/0000-0002-8301-5536


In [5]:
stop_chrome_driver(driver, server, proxy)

# Eliminar los archivos bmp.log y server.log
try:
    os.remove('bmp.log')
    os.remove('server.log')
    print("Se eliminaron los archivos bmp.log y server.log")
except:
    pass

Se eliminaron los archivos bmp.log y server.log
